In [12]:
!pip install nltk

In [13]:
import nltk
nltk.download('punkt_tab')       # Main tokenizer
nltk.download('omw-1.4')     # Optional: for better language support


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [14]:
import json

intents = {
    "intents": [
        {"tag": "greeting", "patterns": ["Hi", "Hello", "Hey"], "responses": ["Hello!", "Hi there!", "Greetings!"]},
        {"tag": "goodbye", "patterns": ["Bye", "See you", "Goodbye"], "responses": ["Bye!", "See you later!", "Goodbye!"]},
        {"tag": "thanks", "patterns": ["Thanks", "Thank you"], "responses": ["You're welcome!", "No problem!"]},
        {"tag": "name", "patterns": ["What is your name?", "Who are you?"], "responses": ["I am your chatbot.", "My name is ChatBot."]},
        {"tag": "help", "patterns": ["Can you help me?", "I need support"], "responses": ["Sure, I'm here to help!", "What can I assist you with?"]}
    ]
}

# Save to a file
with open("intents.json", "w") as file:
    json.dump(intents, file)


In [15]:
import random
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np

# Load intents
with open("intents.json") as file:
    data = json.load(file)

stemmer = PorterStemmer()

# Preprocessing
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(sentence, words):
    sentence_words = [stem(w) for w in tokenize(sentence)]
    bag = [1 if w in sentence_words else 0 for w in words]
    return np.array(bag)


In [16]:
all_words = []
tags = []
xy = []

for intent in data["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    for pattern in intent["patterns"]:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))


In [17]:
def chat():
    print("Start chatting! (type 'quit' to stop)\n")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            print("Chatbot: Goodbye!")
            break

        bow = bag_of_words(inp, all_words)
        scores = []
        for intent in data["intents"]:
            pattern_words = []
            for pattern in intent["patterns"]:
                pattern_words.extend(tokenize(pattern))
            pattern_words = [stem(w) for w in pattern_words]
            match_score = len(set(pattern_words) & set(tokenize(inp)))
            scores.append(match_score)

        max_score = max(scores)
        if max_score == 0:
            print("Chatbot: I don't understand. Try again.")
        else:
            best_tag = data["intents"][scores.index(max_score)]["tag"]
            for intent in data["intents"]:
                if intent["tag"] == best_tag:
                    print("Chatbot:", random.choice(intent["responses"]))


In [19]:
chat()

Start chatting! (type 'quit' to stop)

You: quit
Chatbot: Goodbye!
